# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [11]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
import datetime
# from copy import deepcopy
from sql_queries import *

In [12]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [13]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [80]:
song_files = get_files('data/song_data')

In [81]:
len(song_files)

78

In [ ]:
# for i in range(len(song_files)):
#     df = pd.read_json(song_files[i], typ='series')

#     df.index = ['num_songs', 'artist_id', 'latitude', 'longitude', 'location', 
#                 'name', 'song_id', 'title', 'duration', 'year']
#     print(df)
#     print()

In [56]:
df = pd.read_json(song_files[50], typ='series')

In [57]:
df.index = ['num_songs', 'artist_id', 'latitude', 'longitude', 'location', 
            'name', 'song_id', 'title', 'duration', 'year']
df

num_songs                          1
artist_id         ARIK43K1187B9AE54C
latitude                        None
longitude                       None
location           Beverly Hills, CA
name                   Lionel Richie
song_id           SOBONFF12A6D4F84D8
title        Tonight Will Be Alright
duration                     307.382
year                            1986
dtype: object

In [58]:
# for i in range(len(song_files)):
#     print(pd.read_json(song_files[i], typ='series'))
#     print()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [59]:
sd_vals = ['song_id', 'title', 'artist_id', 'year', 'duration']

In [60]:
song_data = [df.loc[x] for x in sd_vals]
song_data

['SOBONFF12A6D4F84D8',
 'Tonight Will Be Alright',
 'ARIK43K1187B9AE54C',
 1986,
 307.3824]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [61]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [63]:
ad_vals = ['artist_id', 'name', 'location', 'latitude', 'longitude']

In [64]:
artist_data = [df.loc[x] for x in ad_vals]
artist_data

['ARIK43K1187B9AE54C', 'Lionel Richie', 'Beverly Hills, CA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [65]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
# len(log_files)

In [14]:
log_files = get_files('data/log_data')

In [15]:
df = pd.read_json(log_files[0], lines=True)
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df.page == 'NextSong']

In [17]:
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [18]:
# s = 1543449657796 / 1000.0
# datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f')

In [19]:
df['timestamp'] = pd.to_datetime(df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0)))

In [20]:
t = df[['timestamp']]
t.head(30)

,timestamp
0,2018-11-29 00:00:57.796
1,2018-11-29 00:01:30.796
2,2018-11-29 00:04:01.796
3,2018-11-29 00:04:55.796
4,2018-11-29 00:07:13.796
5,2018-11-29 00:10:56.796
6,2018-11-29 00:11:04.796
7,2018-11-29 00:14:10.796
8,2018-11-29 00:16:02.796
9,2018-11-29 00:18:11.796


In [21]:
t.dtypes

timestamp    datetime64[ns]
dtype: object

In [22]:
t.iloc[0].dt.weekday

timestamp    3
Name: 0, dtype: int64

In [25]:
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

lods = []

for i in range(len(t)):
    time_data = (t.iloc[i].dt.time.values[0].minute + round(t.iloc[i].dt.time.values[0].second/60, 3), 
                 t.iloc[i].dt.hour.values[0], 
                 t.iloc[i].dt.day.values[0], 
                 t.iloc[i].dt.week.values[0], 
                 t.iloc[i].dt.month.values[0], 
                 t.iloc[i].dt.year.values[0], 
                 t.iloc[i].dt.weekday.values[0])
    time_dict = {k:v for k, v in list(zip(column_labels, time_data))}
    lods.append(time_dict)

In [27]:
time_df = pd.DataFrame(lods)
time_df = time_df[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df.head(4)

,start_time,hour,day,week,month,year,weekday
0,0.950,0,29,48,11,2018,3
1,1.500,0,29,48,11,2018,3
2,4.017,0,29,48,11,2018,3
3,4.917,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [29]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [30]:
user_df.head(15)

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid
5,24,Layla,Griffin,F,paid
6,73,Jacob,Klein,M,paid
7,73,Jacob,Klein,M,paid
8,24,Layla,Griffin,F,paid
9,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [33]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [34]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-29 00:00:57.796
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24,2018-11-29 00:01:30.796


In [28]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'timestamp'],
      dtype='object')

In [57]:
play_table_create = ("""CREATE TABLE IF NOT EXISTS plays (plays_id SERIAL PRIMARY KEY, timestamp BIGINT, user_id TEXT, 
                                                           level TEXT, song_id TEXT, artist_id TEXT, session_id INT, 
                                                           location TEXT, user_agent TEXT);
""")
play_table_drop = "DROP TABLE IF EXISTS plays;"


In [41]:
# conn.commit()
cur.execute(play_table_create)
conn.commit()

In [42]:
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)
    df = df[df.page == 'NextSong']
        # convert timestamp column to datetime
    df['timestamp'] = pd.to_datetime(df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0)))
    t = df[['timestamp']]
    
    # insert time data records
    column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    lods = []
    for i in range(len(t)):
        time_data = (t.iloc[i].dt.time.values[0].minute + round(t.iloc[i].dt.time.values[0].second/60, 3), 
                     t.iloc[i].dt.hour.values[0], 
                     t.iloc[i].dt.day.values[0], 
                     t.iloc[i].dt.week.values[0], 
                     t.iloc[i].dt.month.values[0], 
                     t.iloc[i].dt.year.values[0], 
                     t.iloc[i].dt.weekday.values[0])
        time_dict = {k:v for k, v in list(zip(column_labels, time_data))}
        lods.append(time_dict)
    time_df = pd.DataFrame(lods)
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    
    for index, row in df.iterrows():

        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.artist, row.song, row.length))
        results = cur.fetchone()

        if results:
            artist_id, song_id = results
            print(results)
        else:
            artist_id, song_id = None, None


    # insert songplay record
    # timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent
        plays_data = (row.ts, int(row.userId), row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
#     print(plays_data)
        cur.execute(play_table_insert, plays_data)
        conn.commit()

('AR5KOSW1187FB35FF4', 'SOZCTXZ12AB0182364')


Run `test.ipynb` to see if you've successfully added records to this table.

In [77]:
df[df.artist == 'Lionel Richie'].head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
294,Lionel Richie,Logged In,Chloe,F,79,Cuevas,310.69995,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,1041,Still,200,1970-01-01 00:25:43.523363796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


# Close Connection to Sparkify Database

In [114]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.